In [4]:
import pandas as pd
train_df = pd.read_csv('ratings_train.txt', sep='\t')
train_df.head(5)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [5]:
train_df['label'].value_counts()

0    75173
1    74827
Name: label, dtype: int64

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [7]:
# null 값은 공백(' ')으로 변환
train_df = train_df.fillna(' ')

In [8]:
import re
# 정규표현식을 이용하여 숫자를 공백으로 변경
train_df['document'] = train_df['document'].apply(lambda x : re.sub(r'\d+',' ', x))
# 불필요한 컬럼은 삭제
train_df.drop('id', axis=1, inplace=True)

In [9]:
# 테스트 데이터 셋을 로딩하고 동일하게 null 및 숫자를 공백으로 전환
test_df = pd.read_csv('ratings_test.txt', sep='\t')
test_df = test_df.fillna(' ')
test_df['document'] = test_df['document'].apply(lambda x : re.sub(r'\d+',' ', x) )
test_df.drop('id', axis=1, inplace=True)

In [10]:
import konlpy

In [11]:
from konlpy.tag import Twitter
# 한글을 토큰화하는 Twitter

In [12]:
twitter = Twitter()
# 파이썬과 jpype버전 충돌

C:\Users\204\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
def tw_tokenizer(text):
    # 입력받은 text를 형태소로 분해 토큰화하여 list로 리턴
    tokens_ko = twitter.morphs(text)
    return tokens_ko
# 문장 내 단어 조사 합성어의 단어별 구분을 적용하여 단어분리기능을 수행합니다

In [15]:
tw_tokenizer('포스터보고 초딩영화인줄....오버연기조차 가볍지 않구나')

['포스터', '보고', '초딩', '영화인', '줄', '....', '오버', '연기', '조차', '가볍지', '않구나']

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [29]:
# min_df=3 : 3회 이하 출현 단어 제외
# max_df=0.9 : 총 문건중 출현 비율 90이상되는 단어 제외
# TfidVectorizer에 단어분리도구로써 tw_tokenize 함수를 제공
tfidf_vect = TfidfVectorizer(tokenizer=tw_tokenizer, ngram_range=(1,2), min_df=3,\
                                                                     max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

In [30]:
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

In [32]:
from sklearn.model_selection import GridSearchCV
ln_clf = LogisticRegression(random_state=0)
params = {'C':[4.5, 5.5, 10]}
grid_cv = GridSearchCV(ln_clf, param_grid=params, cv=3, scoring='accuracy')
grid_cv.fit(tfidf_matrix_train, train_df['label'])

GridSearchCV(cv=3, estimator=LogisticRegression(random_state=0),
             param_grid={'C': [4.5, 5.5, 10]}, scoring='accuracy')

In [33]:
pred = grid_cv.predict(tfidf_matrix_test)

In [34]:
from sklearn.metrics import accuracy_score
print('정확도 : {0:.3f}'.format(accuracy_score(test_df['label'], pred)))

정확도 : 0.862
